खालील नोटबुक चालवण्यासाठी, जर तुम्ही अजून केले नसेल, तर तुम्हाला `text-embedding-ada-002` या बेस मॉडेलचा वापर करणारे मॉडेल डिप्लॉय करावे लागेल आणि .env फाइलमध्ये डिप्लॉयमेंटचे नाव `AZURE_OPENAI_EMBEDDINGS_ENDPOINT` असे सेट करावे लागेल.


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

पुढे, आपण Embedding Index ला Pandas Dataframe मध्ये लोड करणार आहोत. Embedding Index `embedding_index_3m.json` नावाच्या JSON फाईलमध्ये संग्रहित आहे. Embedding Index मध्ये ऑक्टोबर २०२३ च्या अखेरीपर्यंतच्या प्रत्येक YouTube ट्रान्सक्रिप्टसाठी Embeddings आहेत.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

पुढे, आपण `get_videos` नावाची एक फंक्शन तयार करणार आहोत जी क्वेरीसाठी Embedding Index मध्ये शोध घेईल. ही फंक्शन क्वेरीशी सर्वात जास्त मिळती-जुळती अशी टॉप ५ व्हिडिओ परत करेल. ही फंक्शन खालीलप्रमाणे काम करते:

1. सर्वप्रथम, Embedding Index ची एक प्रत तयार केली जाते.
2. नंतर, क्वेरीसाठी Embedding OpenAI Embedding API वापरून काढली जाते.
3. त्यानंतर Embedding Index मध्ये `similarity` नावाचा एक नवीन कॉलम तयार केला जातो. `similarity` कॉलममध्ये क्वेरी Embedding आणि प्रत्येक व्हिडिओ सेगमेंटसाठी Embedding यांच्यातील कोसाइन सिमिलॅरिटी असते.
4. पुढे, Embedding Index ला `similarity` कॉलमने फिल्टर केले जाते. Embedding Index मध्ये फक्त तेच व्हिडिओ ठेवले जातात ज्यांची कोसाइन सिमिलॅरिटी 0.75 किंवा त्याहून जास्त आहे.
5. शेवटी, Embedding Index ला `similarity` कॉलमने सॉर्ट केले जाते आणि टॉप ५ व्हिडिओ परत केले जातात.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

ही फंक्शन खूप सोपी आहे, ती फक्त शोध क्वेरीचे परिणाम छापते.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. प्रथम, Embedding Index एक Pandas Dataframe मध्ये लोड केला जातो.
2. पुढे, वापरकर्त्याला एक क्वेरी टाकण्यास सांगितले जाते.
3. नंतर `get_videos` फंक्शन कॉल करून Embedding Index मध्ये त्या क्वेरीसाठी शोध घेतला जातो.
4. शेवटी, `display_results` फंक्शन कॉल करून वापरकर्त्याला निकाल दाखवले जातात.
5. त्यानंतर वापरकर्त्याला पुन्हा एक क्वेरी टाकण्यास सांगितले जाते. हा प्रक्रिया वापरकर्ता `exit` टाकेपर्यंत सुरू राहते.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.mr.png)

तुम्हाला क्वेरी टाकण्यास सांगितले जाईल. एक क्वेरी टाका आणि enter दाबा. अ‍ॅप्लिकेशन त्या क्वेरीसाठी संबंधित व्हिडिओंची यादी परत करेल. तसेच, प्रश्नाचे उत्तर ज्या ठिकाणी व्हिडिओमध्ये आहे त्या ठिकाणाचा दुवा देखील अ‍ॅप्लिकेशन परत करेल.

हे काही क्वेरी आहेत ज्या तुम्ही वापरून पाहू शकता:

- Azure Machine Learning म्हणजे काय?
- convolutional neural networks कसे काम करतात?
- neural network म्हणजे काय?
- मी Azure Machine Learning सोबत Jupyter Notebooks वापरू शकतो का?
- ONNX म्हणजे काय?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**अस्वीकरण**:  
हे दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) वापरून भाषांतरित केले आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात घ्या की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील मूळ दस्तऐवज हा अधिकृत स्रोत मानावा. अत्यावश्यक माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर केल्यामुळे उद्भवणाऱ्या कोणत्याही गैरसमजुती किंवा चुकीच्या अर्थ लावण्यास आम्ही जबाबदार राहणार नाही.
